## Viz complémentaire axe 2.3

Silvia 16/05 - "Thierry, que en penses tu de demander une carte des destinations de navires chargés du sel (tonnage total?) sortis de Marennes pour explorer la question des marchés et des acteurs ?
avec possibilité de choisir pavillon français / pavillon étranger ?"

Nous allons essayer d'utiliser à la fois les données Toflit et les données Navigo :

- pointcalls Navigo : pointcal_action = 'Out', year = 1789, ferme_bureau = 'Marennes' (ou pointcall = 'Marennes', 'Ribérou', 'La Perrotine', 'Saint Denis d'Oléron', 'Le Chateau d' Oléron', 'Soubise', 'Saint Savinien', 'Royan', 'Anveaux'), je peux regarder ship_flag_standardized_fr pour le pavillon

**=> que puis je faire pour avoir la bonne destination ? à priori je ne peux pas utiliser les flows pour 1789**

- flows Toflit : year = 1789, customs_office = 'Marennes'

**(pour le pavillon je peux regarder partner_revolutionempire ?? ce n'est pas le pavillon à proprement parler)**


In [4]:
# Load an empty map
from keplergl import KeplerGl
from poitousprint import Portic, Toflit, combine_commodity_purposes, get_pointcalls_commodity_purposes_as_toflit_product

portic_client = Portic()
toflit_client = Toflit()

In [2]:
pointcalls_navigo = portic_client.get_pointcalls(
    year = 1789, 
    ferme_bureau = 'Marennes' # après vérification, tous les pointcalls 'Out' de 1789 qui concernent les ports rattachés au bureau des Fermes de Marennes ont bien l'attribut ferme_bureau à 'Marennes'
)

print("nombre de pointcalls sortant de Marennes avec du sel :", len(pointcalls_navigo))

nombre de pointcalls sortant de Marennes avec du sel : 2535


In [14]:
pointcall_test = pointcalls_navigo[0] 
# verifier ce à quoi peut ressembler un pointcall correspondant
combine_commodity_purposes(pointcall_test)

{'pkid': 77030,
 'record_id': '00333308',
 'pointcall': 'Ribérou [Saujon]',
 'pointcall_uhgs_id': 'A1964767',
 'toponyme_fr': 'Ribérou',
 'toponyme_en': 'Ribérou',
 'latitude': '45.6755',
 'longitude': '-0.935826',
 'pointcall_admiralty': 'Marennes',
 'pointcall_province': 'Saintonge',
 'pointcall_states': '[{"1749-1815" : "France"}]',
 'pointcall_substates': None,
 'pointcall_states_en': '[{"1749-1815" : "France"}]',
 'pointcall_substates_en': None,
 'state_1789_fr': 'France',
 'state_1789_en': 'France',
 'substate_1789_fr': None,
 'substate_1789_en': None,
 'nb_conges_1787_inputdone': 480.0,
 'nb_conges_1787_cr': 480.0,
 'nb_conges_1789_inputdone': 434.0,
 'nb_conges_1789_cr': 434.0,
 'pointcall_status_uncertainity': None,
 'pointcall_status': 'oblique',
 'shiparea': 'ACE-ROCH',
 'pointcall_point': '0101000020110F0000CA2ED9C7FA6EF9C0334934DF3CDA5541',
 'ferme_direction': 'La Rochelle',
 'ferme_bureau': 'Marennes',
 'ferme_bureau_uncertainty': 0.0,
 'partner_balance_1789': None,
 'par

In [16]:
pointcalls_navigo_with_revolution_empire_products = get_pointcalls_commodity_purposes_as_toflit_product(pointcalls_navigo, product_classification="product_revolutionempire")
print(pointcalls_navigo_with_revolution_empire_products[100:110])

[{'pkid': 82182, 'record_id': '00338460', 'pointcall': 'Marennes', 'pointcall_uhgs_id': 'A0136930', 'toponyme_fr': 'Marennes', 'toponyme_en': 'Marennes', 'latitude': '45.816667', 'longitude': '-1.116667', 'pointcall_admiralty': 'Marennes', 'pointcall_province': 'Saintonge', 'pointcall_states': '[{"1749-1815" : "France"}]', 'pointcall_substates': None, 'pointcall_states_en': '[{"1749-1815" : "France"}]', 'pointcall_substates_en': None, 'state_1789_fr': 'France', 'state_1789_en': 'France', 'substate_1789_fr': None, 'substate_1789_en': None, 'nb_conges_1787_inputdone': 1172.0, 'nb_conges_1787_cr': 1172.0, 'nb_conges_1789_inputdone': 1481.0, 'nb_conges_1789_cr': 1480.0, 'pointcall_status_uncertainity': 0.0, 'pointcall_status': 'siège amirauté', 'shiparea': 'ACE-ROCH', 'pointcall_point': '0101000020110F0000CC2247D42C59FEC031FEA89F3AF05541', 'ferme_direction': 'La Rochelle', 'ferme_bureau': 'Marennes', 'ferme_bureau_uncertainty': 0.0, 'partner_balance_1789': None, 'partner_balance_supp_1789'

In [17]:
# filtrer les pointcalls pour lesquels la cargaison contient du sel
final_pointcalls_navigo = []

for pointcall in pointcalls_navigo_with_revolution_empire_products:
    for commodity_purpose in pointcall['commodity_purposes']:
        if (commodity_purpose['commodity_as_toflit'] == 'Sel'):
            final_pointcalls_navigo.append(pointcall)
        
print("Nombre de pointcalls pertinents trouvés ('Out' de Marennes en 1789 et transportant du sel) :", len(final_pointcalls_navigo))

Nombre de pointcalls pertinents trouvés ('Out' de Marennes en 1789 et transportant du sel) : 1310


In [2]:
# tentative d'obtenir un nest_portic_pointcall qui fonctionne (même s'il oublie certains arguments)
fieldnames = portic_client.get_fieldnames()

fieldnames[0:5]

[{'api': 'None',
  'name': 'travel_id',
  'shortname': 't001',
  'type': 'text',
  'description': 'Identifiant du trajet'},
 {'api': 'None',
  'name': 'distance_dep_dest',
  'shortname': 't002',
  'type': 'numeric',
  'description': "Distance en km entre départ et destination, arrondie à 3 decimales, à vol d'oiseau"},
 {'api': 'None',
  'name': 'distance_homeport_dep',
  'shortname': 't003',
  'type': 'numeric',
  'description': "Distance en km entre homeport et départ, arrondie à 3 decimales, à vol d'oiseau"},
 {'api': 'None',
  'name': 'departure',
  'shortname': 't004',
  'type': 'text',
  'description': 'Toponyme du port de départ'},
 {'api': 'None',
  'name': 'departure_fr',
  'shortname': 't005',
  'type': 'text',
  'description': 'Standardised name of the port de départ, in French'}]

In [3]:
fieldnames_list = []
for field in fieldnames:
    fieldnames_list.append(field['name'])
    
fieldnames_list

['travel_id',
 'distance_dep_dest',
 'distance_homeport_dep',
 'departure',
 'departure_fr',
 'departure_en',
 'departure_uhgs_id',
 'departure_latitude',
 'departure_longitude',
 'departure_admiralty',
 'departure_province',
 'departure_states',
 'departure_substates',
 'departure_state_1789_fr',
 'departure_substate_1789_fr',
 'departure_state_1789_en',
 'departure_substate_1789_en',
 'departure_ferme_direction',
 'departure_ferme_bureau',
 'departure_ferme_bureau_uncertainty',
 'departure_partner_balance_1789',
 'departure_partner_balance_supp_1789',
 'departure_partner_balance_1789_uncertainty',
 'departure_partner_balance_supp_1789_uncertainty',
 'departure_shiparea',
 'departure_status',
 'departure_nb_conges_1787_inputdone',
 'departure_nb_conges_1787_cr',
 'departure_nb_conges_1789_inputdone',
 'departure_nb_conges_1789_cr',
 'departure_status_uncertainity',
 'departure_point',
 'departure_out_date',
 'departure_action',
 'outdate_fixed',
 'departure_navstatus',
 'departure_fun